## Transform the data
In this notebook we transformed the data into the star schema for a Gold data store

In [0]:
# Create a new schema
spark.sql('''
          CREATE SCHEMA IF NOT EXISTS gold
          ''')

Out[1]: DataFrame[]

#### Dimension Riders

In [0]:
# Load the data from the table
riders_df = spark.read.table("staging.riders")
riders_df.show(2)

+--------+----------+---------+-------------------+----------+------------------+----------------+---------+
|rider_id|first_name|last_name|            address|  birthday|account_start_date|account_end_date|is_member|
+--------+----------+---------+-------------------+----------+------------------+----------------+---------+
|    1000|     Diana|    Clark|1200 Alyssa Squares|1989-02-13|        2019-04-23|            null|     true|
|    1001|  Jennifer|    Smith|    397 Diana Ferry|1976-08-10|        2019-11-01|      2020-09-01|     true|
+--------+----------+---------+-------------------+----------+------------------+----------------+---------+
only showing top 2 rows



In [0]:
# create the dimension table
riders_df.dropDuplicates(["rider_id"]).write.format("delta").mode("overwrite").saveAsTable("gold.dim_riders")

In [0]:
# Verify the data
spark.sql('''
          SELECT * FROM gold.dim_riders;
          ''').show(2)

+--------+----------+---------+--------------------+----------+------------------+----------------+---------+
|rider_id|first_name|last_name|             address|  birthday|account_start_date|account_end_date|is_member|
+--------+----------+---------+--------------------+----------+------------------+----------------+---------+
|    1005| Christine|Rodriguez|224 Washington Mi...|1974-08-27|        2020-03-24|            null|    false|
|    1008|      John| Crawford|    7691 Evans Court|1987-02-21|        2021-03-28|      2021-07-01|     true|
+--------+----------+---------+--------------------+----------+------------------+----------------+---------+
only showing top 2 rows



#### Dimension Stations

In [0]:
# Load the data from the table
stations_df = spark.read.table("staging.stations")
stations_df.show(2)

+------------+--------------------+-----------------+------------------+
|  station_id|                name|         latitude|         longitude|
+------------+--------------------+-----------------+------------------+
|         525|Glenwood Ave & To...|        42.012701|-87.66605799999999|
|KA1503000012|  Clark St & Lake St|41.88579466666667|-87.63110066666668|
+------------+--------------------+-----------------+------------------+
only showing top 2 rows



In [0]:
# create the station dimension table
stations_df.dropDuplicates(["station_id"]) \
    .withColumnRenamed("name", "station_name") \
    .write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.dim_stations")

In [0]:
# Verify the data
spark.sql('''
          SELECT * FROM gold.dim_stations;
          ''').show(2)

+----------+--------------------+----------------+-----------------+
|station_id|        station_name|        latitude|        longitude|
+----------+--------------------+----------------+-----------------+
|     13001|Michigan Ave & Wa...|41.8839840647265|-87.6246839761734|
|     13006|LaSalle St & Wash...|       41.882664|        -87.63253|
+----------+--------------------+----------------+-----------------+
only showing top 2 rows



#### Dimension Dates

In [0]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
# Load the data from the table
payments_df = spark.read.table("staging.payments")
payments_df.show(2)

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|         1|2019-05-01|   9.0|    1000|
|         2|2019-06-01|   9.0|    1000|
+----------+----------+------+--------+
only showing top 2 rows



In [0]:
# Create a partition by date
w = Window.partitionBy('date').orderBy('date')

In [0]:
unique_dates_df = (payments_df.withColumn('rank',f.row_number().over(w)))
unique_dates_df = (unique_dates_df.filter(unique_dates_df['rank'] == 1).drop('rank'))

In [0]:
# Let's diplay 2 rows from the dataframe
unique_dates_df.show(2)

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|      1890|2013-08-01|   6.7|    1072|
|      1363|2014-05-01| 22.86|    1052|
+----------+----------+------+--------+
only showing top 2 rows



In [0]:

# Create the dim_dates dataframe
dim_dates_df = unique_dates_df \
    .withColumn("date_id", regexp_replace(unique_dates_df.date, "-", "").cast(IntegerType())) \
    .withColumn("day", f.dayofweek(unique_dates_df.date)) \
    .withColumn("week", f.weekofyear(unique_dates_df.date)) \
    .withColumn("month", f.month(unique_dates_df.date)) \
    .withColumn("quarter", f.quarter(unique_dates_df.date)) \
    .withColumn("year", f.year(unique_dates_df.date)) \
    .withColumn("is_weekend", f.dayofweek(unique_dates_df.date).isin([1,7]).cast("int"))

In [0]:
# Let's see the result
dim_dates_df_ = dim_dates_df[["date_id", "day", "week", "month", "quarter", "year", "is_weekend"]]
dim_dates_df_.show(2)

+--------+---+----+-----+-------+----+----------+
| date_id|day|week|month|quarter|year|is_weekend|
+--------+---+----+-----+-------+----+----------+
|20130201|  6|   5|    2|      1|2013|         0|
|20130301|  6|   9|    3|      1|2013|         0|
+--------+---+----+-----+-------+----+----------+
only showing top 2 rows



In [0]:
# create the dimension table
dim_dates_df_.dropDuplicates(["date_id"]).write.format("delta").mode("overwrite").saveAsTable("gold.dim_dates")


In [0]:
# Verify the data
spark.sql('''
          SELECT * FROM gold.dim_dates;
          ''').show(2)

+--------+---+----+-----+-------+----+----------+
| date_id|day|week|month|quarter|year|is_weekend|
+--------+---+----+-----+-------+----+----------+
|20180301|  5|   9|    3|      1|2018|         0|
|20200801|  7|  31|    8|      3|2020|         1|
+--------+---+----+-----+-------+----+----------+
only showing top 2 rows



### Fact Payment

In [0]:
# Load the data from the table
payments_df = spark.read.table("staging.payments")
payments_df.show(2)

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|         1|2019-05-01|   9.0|    1000|
|         2|2019-06-01|   9.0|    1000|
+----------+----------+------+--------+
only showing top 2 rows



In [0]:
# create the payments fact table
payments_df.dropDuplicates(["payment_id"]) \
    .write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.fact_payments")

In [0]:
# Verify the data
spark.sql('''
          SELECT * FROM gold.fact_payments;
          ''').show(2)

+----------+----------+------+--------+
|payment_id|      date|amount|rider_id|
+----------+----------+------+--------+
|       148|2017-09-01|  6.86|    1007|
|       463|2021-09-01| 22.35|    1018|
+----------+----------+------+--------+
only showing top 2 rows



#### Fact Trips

For Fact Trips table we used spark.sql with a SQL query, we could have used pyspark.sql.DataFrame.join but it 's more work and we would need to upload the other tables in different dataframe

In [0]:
# Load the data from the table
trips_df = spark.read.table("staging.trips")
trips_df.show(2)

+----------------+-------------+-------------------+-------------------+----------------+--------------+--------+
|         trip_id|rideable_type|           start_at|           ended_at|start_station_id|end_station_id|rider_id|
+----------------+-------------+-------------------+-------------------+----------------+--------------+--------+
|89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43|             525|           660|   71934|
|0FEFDE2603568365| classic_bike|2021-02-14 17:52:38|2021-02-14 18:12:09|             525|         16806|   47854|
+----------------+-------------+-------------------+-------------------+----------------+--------------+--------+
only showing top 2 rows



In [0]:
trips_df.count()

Out[32]: 4584921

In [0]:
# Load the data into a dataframe
fact_trips_df = spark.sql('''
                SELECT
                    t.trip_id,
                    t.rideable_type,
                    t.start_at,
                    t.ended_at,
                    DATEDIFF(MINUTE, t.start_at, t.ended_at) AS Duration,
                    DATEDIFF(YEAR, r.birthday, t.ended_at) AS rider_age,
                    t.start_station_id,
                    t.end_station_id,
                    t.rider_id,
                    p.date date_id
                FROM staging.trips t
                JOIN staging.riders r ON (t.rider_id=r.rider_id)
                JOIN staging.payments p ON (t.rider_id=p.rider_id)
            ''')

In [0]:
fact_trips_df.count()

Out[30]: 117816784

From 4584921 to 117816784 is a lot, that's mean we have some duplicated data

In [0]:
# Let's display the first 10 rows to verify
fact_trips_df.show(10)

+----------------+-------------+-------------------+-------------------+--------+---------+----------------+--------------+--------+----------+
|         trip_id|rideable_type|           start_at|           ended_at|Duration|rider_age|start_station_id|end_station_id|rider_id|   date_id|
+----------------+-------------+-------------------+-------------------+--------+---------+----------------+--------------+--------+----------+
|89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43|       6|       37|             525|           660|   71934|2022-02-01|
|89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43|       6|       37|             525|           660|   71934|2022-01-01|
|89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43|       6|       37|             525|           660|   71934|2021-12-01|
|89E7AA6C29227EFF| classic_bike|2021-02-12 16:14:56|2021-02-12 16:21:43|       6|       37|             525|           660|   71934|2021

We have noticed that we have some dupricates rows but we will drop the duplicates data before we insert them into the table.

In [0]:
# create the trips fact table
fact_trips_df.dropDuplicates(["trip_id"]) \
    .write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.fact_trips")

In [0]:
# Verify the data and check if we still have the duplicates data
spark.sql('''
          SELECT * FROM gold.fact_trips;
          ''').show(10)

+----------------+-------------+-------------------+-------------------+--------+---------+----------------+--------------+--------+----------+
|         trip_id|rideable_type|           start_at|           ended_at|Duration|rider_age|start_station_id|end_station_id|rider_id|   date_id|
+----------------+-------------+-------------------+-------------------+--------+---------+----------------+--------------+--------+----------+
|00000CAE95438C9D| classic_bike|2021-07-20 15:40:46|2021-07-20 17:38:17|     117|       22|           13022|  TA1305000003|   71748|2022-02-01|
|00001DCF2BC423F4|  docked_bike|2021-06-13 12:00:49|2021-06-13 12:29:51|      29|       39|           13008|  TA1307000048|   21478|2022-02-01|
|00002E8260690FFF| classic_bike|2021-09-29 17:43:26|2021-09-29 17:49:06|       5|       33|           13193|  TA1309000058|   23449|2022-02-01|
|0000578C9F82736A| classic_bike|2021-10-06 22:31:26|2021-10-06 23:03:50|      32|       42|           13257|  KA1503000041|   46411|2022

In [0]:
# Check how many record we have in the table
spark.sql('''
          SELECT * FROM gold.fact_trips;
          ''').count()

Out[36]: 4463338